## 👨🏻‍💻 Практика - Масштабирование признаков
## 👨🏻‍💻 Практика - Заполнение пропущенных значений
![](../images/01_01.png)

✔️**Решение:**
Использован датасет Titanic 
### ✔ Импорт библиотек: 
Импортируем все необходимые библиотеки

In [1]:
import pandas as pd
# графики
import matplotlib.pyplot as plt
import seaborn as sns
# для разделения данных
from sklearn.model_selection import train_test_split
# для создания модели
from sklearn.linear_model import LinearRegression, LogisticRegression
# для оценки качества
from sklearn.metrics import mean_squared_error, accuracy_score
# для стандартизации набора данных(вычитание ср.знач.и деления на станд.откл)
# программирование категориальных переменных как двоичных векторов.
from sklearn.preprocessing import StandardScaler, OneHotEncoder, MinMaxScaler
# Восстановление (импутация) данных с помощью Python
from sklearn.impute import SimpleImputer
# преобразования к определённым столбцам внутри набора данных
from sklearn.compose import ColumnTransformer 
# для создания конвейера шагов предв.обработки данных и моделирования.
from sklearn.pipeline import Pipeline

### ✔1. Загрузка и подготовка данных: 
Загружаем данные - запишем в датафрейм датасет Titanic из библиотеки сиборна.

In [10]:
data = sns.load_dataset("titanic")
data.head()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True


In [11]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 15 columns):
 #   Column       Non-Null Count  Dtype   
---  ------       --------------  -----   
 0   survived     891 non-null    int64   
 1   pclass       891 non-null    int64   
 2   sex          891 non-null    object  
 3   age          714 non-null    float64 
 4   sibsp        891 non-null    int64   
 5   parch        891 non-null    int64   
 6   fare         891 non-null    float64 
 7   embarked     889 non-null    object  
 8   class        891 non-null    category
 9   who          891 non-null    object  
 10  adult_male   891 non-null    bool    
 11  deck         203 non-null    category
 12  embark_town  889 non-null    object  
 13  alive        891 non-null    object  
 14  alone        891 non-null    bool    
dtypes: bool(2), category(2), float64(2), int64(4), object(5)
memory usage: 80.7+ KB


### ✔ 2. Разделение на признаки и целевую переменную: 
Указываем, какие столбцы в данных являются признаками (X) и какой столбец - целевой переменной (y).  
Предположим, что 'survived' - выживший (0 - не выжил, 1 - выжил), а остальные столбцы - признаки - отберем только несколько столбов из датасета

In [12]:
# Характеристики и целевая переменная
X = data[['pclass', 'sex', 'age', 'sibsp', 'parch', 'fare']]
y = data['survived']

In [13]:
# Удаляем строки с отсутствующими "Survived" значениями
data = data.dropna(subset=['survived'])

# кодируем столбец  'Sex'
X.loc[:, 'sex'] = X['sex'].map({'female': 0, 'male': 1})

# Заполняем недостающие значения "Возраста" медианой
median_value = X['age'].median()
X['age'] = X['age'].fillna(median_value) 

### ✔ 3. Разделение на обучающую и тестовую выборки: 
Разделяем данные на обучающую и тестовую выборки в соотношении 80/20. 

random_state используется для воспроизводимости результатов.

In [14]:
X_train, X_test, y_train, y_test = (
    train_test_split(X, y, 
                     test_size=0.2, 
                     random_state=1))

### ✔4. Создание конвейера предобработки данных
Используем MinMaxScaler() для масштабирования числовых признаков в диапазон 0, 1.

#### ✔ 4.1  Числовые признаки
Запишем в переменные списки с названиями столбцов с числовыми и категориальными данными из датасета

In [15]:
# Числовые признаки
numeric_features = ['age','fare']
# Используем MinMaxScaler() для масштабирования числовых признаков в диапазон 0, 1.
numeric_transformer = Pipeline(steps =[
    ('imputer',SimpleImputer(strategy='mean')),
    ('scaler',MinMaxScaler())     
])

#### ✔ 4.2  Категориальные признаки 

In [16]:
# Категориальные признаки 
categorical_features = ['pclass','sex']
# Категориальные признаки (заполнение пропусков, one-hot encoding)
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

#### ✔ 4.3 Объединение всех преобразований

In [17]:
# Объединение всех преобразований
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ])

###  ✔ 5.Создание конвейера модели (логистическая регрессия)


In [18]:
# логистическая регрессия
model_pipeline_logreg = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('model', LogisticRegression())
])

### ✔ 6 Обучение и оценка моделей для логистической регрессий
#### ✔ 6.1 Логистическая регрессия - обучение и оценка на необработанных данных


In [19]:
# Логистическая регрессия - обучение и оценка на необработанных данных

model_logreg_no_preprocessing = LogisticRegression()
model_logreg_no_preprocessing.fit(X_train, y_train)

y_pred_logreg_no_preprocessing = (
    model_logreg_no_preprocessing.predict(X_test))

accuracy_logreg_no_preprocessing = (
    accuracy_score(y_test, y_pred_logreg_no_preprocessing))

print(f"Accuracy (Логистическая регрессия без обработки\
): {accuracy_logreg_no_preprocessing:.4f}")

Accuracy (Логистическая регрессия без обработки): 0.7989


#### ✔ 6.2 Логистическая регрессия - обучение и оценка на обработанных данных

In [22]:
# Логистическая регрессия - обучение и оценка на обработанных данных

model_logreg = model_pipeline_logreg.fit(X_train, y_train)
y_pred_logreg = model_logreg.predict(X_test)
accuracy_logreg = accuracy_score(y_test, y_pred_logreg)
print(f"Accuracy (Логистическая регрессия с обработкой\
): {accuracy_logreg:.4f}")

Accuracy (Логистическая регрессия с обработкой): 0.7877


### 7 ✔ Сравнение результатов

#### ✔ Сравнение метрик модели  логистической регрессий
Сравниваем метрики для моделей с масштабированием и без него, чтобы оценить влияние масштабирования признаков.
    

In [24]:
if accuracy_logreg_no_preprocessing > accuracy_logreg:
    print(f"Модель без масштабирования лучше: \
Accuracy {accuracy_logreg_no_preprocessing:.4f} > {accuracy_logreg:.4f}")
elif accuracy_logreg_no_preprocessing < accuracy_logreg:
    print(f"Модель с масштабированием лучше: \
Accuracy {accuracy_logreg:.4f} > {accuracy_logreg_no_preprocessing:.4f}")
else:
    print(f"Обе модели имеют одинаковую точность\
Accuracy: {accuracy_logreg_no_preprocessing:.4f} = {accuracy_logreg_no_preprocessing:.4f}")

Модель без масштабирования лучше: Accuracy 0.7989 > 0.7877
